# Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
# Date; 3. 7. 2022 - 3. . 2022
# Title: Korean_NER
# Professor: Seung-Hoon Na

In [308]:
import os
import json
from functools import partial
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, FlaubertModel
# from transformers import BertModel
from sklearn.metrics import classification_report

In [57]:
from DataPreprocessing import Data_NER, ner_collate_fn
from modeling_ner import Bert_NER

ModuleNotFoundError: No module named 'DataPreprocessing'

In [201]:
from ner_collate_fn import collate_fn_custom

In [310]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)
bert = BertModel.from_pretrained(PRETAINED_MODEL_NAME)
# flaubert = FlaubertModel.from_pretrained(language_model_dir, output_loading_info=True)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# BertTokenizer
 - return tensors = "pt" : Finally, you want the tokenizer to return the actual tensors that are fed to the model.
 
# BertModel
 - Attention_mask : 1 where you care and 0 where you don't care.
 - Input_ids : the IDs of the sentence morpheme.
 - Token_type_ids : for the question problem, but it's enough to set it to zero now.

reference: https://huggingface.co/docs/transformers/model_doc/bert

# Example

# Loading data

In [7]:
PATH_dir = 'C:\\Users\\LG\\Desktop\\github\\JBNU-2022-SPRING\\English world class tagging & Korean_Named Entity Recognition\\Ko_En_NER_POStag_data\Ko_NER_POS'
PATH_ko_train = os.path.join(PATH_dir, 'prepro_train.json')
PATH_ko_test = os.path.join(PATH_dir, 'prepro_test.json')
PATH_ko_dev = os.path.join(PATH_dir, 'prepro_dev.json')
total_tag = os.path.join(PATH_dir, 'total_tag.json')

In [11]:
with open(PATH_ko_train) as f: 
    dataset_train = json.load(f)
with open(PATH_ko_test) as f: 
    dataset_test = json.load(f)
with open(PATH_ko_dev) as f: 
    dataset_dev = json.load(f)
with open(total_tag, 'r') as f:
    tag_converter = json.load(f)

In [12]:
print('train', len(dataset_train))
print('test', len(dataset_test))
print('dev', len(dataset_dev))
# tag_converter.id_to_tag

train 4250
test 500
dev 250


# collate_fn
 - Tie it in batches.
 - If the data set is variable length, it cannot be tied right away and causes an error, so you have to make a 'collate_fn' and hand it over.
  - partial_collate_fn: when collating, with padding
  
  
  - reference: https://hulk89.github.io/pytorch/2019/09/30/pytorch_dataset/

# collate_fn_custom

In [120]:
def collate_fn_custom(data):
    
    input_sent = [sample[0] for sample in data]
    labels = [sample[2] for sample in data]
    
    padded_inputs = torch.nn.utils.rnn.pad_sequence(input_sent, batch_first = True)
    
    return {'input': padded_inputs.contiguous(),
            'label': torch.stack(labels).contiguous()}

# 0309

In [273]:
from ner_collate_fn import make_same_len

In [274]:
def ner_fn_custom(tokenizer, data):
    
    input_sent = [sample[0] for sample in data]
    labels = [sample[2] for sample in data]

    batch_inputs = tokenizer(input_sent, padding = True, return_tensors = "pt")
    batch_labels = make_same_len(labels)
#     batch_labels = torch.tensor(batch_labels)
    
    return batch_inputs, batch_labels
#     return {'input': padded_inputs.contiguous(),
#             'label': torch.stack(labels).contiguous()}

In [163]:
def make_batch(labels, max_len):
    batch_labels = []
    special_token = 0
    
    for 

SyntaxError: invalid syntax (<ipython-input-163-f11b6230085a>, line 3)

In [164]:
batch_size = 10

In [275]:
batch_size = 1
# partial (func, /, *args, **keywords)  positional argument: args, keyword argument: keywords

# partial_collate_fn = partial(collate_fn_custom, tokenizer, tag_converter)
# partial_collate_fn = collate_fn_custom

partial_collate_fn = partial(ner_fn_custom, tokenizer)

'''
partial_collate_fn = partial(collate_fn_custom, tokenizer, tag_converter)
'''
# ner_collate_fn: padding & making batch?
'''
def ner_collate_fn(tokenizer, tag_converter):
    
'''

'\ndef ner_collate_fn(tokenizer, tag_converter):\n    \n'

In [276]:
dataloader_train = DataLoader(
    dataset_train,
    batch_size = batch_size,
    shuffle = True,
    collate_fn = partial_collate_fn)

dataloader_test = DataLoader(
    dataset_test,
    batch_size = batch_size,
    shuffle = False,
    collate_fn = partial_collate_fn)

dataloader_dev = DataLoader(
    dataset_dev,
    batch_size = batch_size,
    shuffle = False,
    collate_fn = ner_fn_custom)

In [139]:
for data in dataloader_dev:
    print(data)

TypeError: ner_fn_custom() missing 1 required positional argument: 'data'

# BERT_NER
 - Dropout: method to solve overfitting, one of the problems in deep learning learning. In summary, how to prevent some units of the hidden layer from being overfitted by inoperative<br>
    ->the outputs are scaled by a factor of (1 / 1-p)  during training

# *args, **kargs
 - *args(non-keworded arguments): Tuple type of argument without limitation.
 - **kargs(keworded arguments):  Dictionary type of argument without limitation.

In [316]:
class Bert_NER(nn.Module):
    
    def __init__(self, bert):
        super().__init__()        
        self.bert = bert
        self.dorpout = nn.Dropout(p = 0.1)
        self.linear = nn.Linear(768, 22)
        self.softmax = nn.Softmax(dim = 2) # A dimension along which Softmax will be computed
        
    def forward(self, **kwargs):
        emb = self.bert(**kwargs)
        e = self.dropout(emb['last_hidden_state'])
        w = self.linear(e)
        
        return w

In [294]:
tag_to_id ={'[PAD]': 0, 'B-<휠': 1, 'B-OG': 2, 'I-조선': 3, 'I-PS': 4, 'B-LC': 5, 'B-1': 6, 'I-<휠': 7, 'I-LC': 8, 'B-PS': 9, 'I-TI': 10, 
            'B-목소': 11, 'O': 12, 'I-목소': 13, 'I-1': 14, 'I-': 15, 'B-조선': 16, 'I-OG': 17, 'B-': 18, 'I-DT': 19, 'B-TI': 20, 'B-DT': 21}

In [295]:
len(tag_to_id)

22

In [312]:
tag_num = len(tag_to_id) # 22
model = Bert_NER(22)

In [313]:
CELoss = nn.CrossEntropyLoss(ignore_index = 0)
optimizer = AdamW(model.parameters(), lr = 1.0e-5)

In [314]:
# model.cuda(6)
# device = model.bert.device
# print(device)

# START!~

# Sample

In [315]:
for epoch in range(3): # 숫자 ->train_epoch
    model.train()
    
    for iteration, batch in enumerate(dataloader_train):
        batch_inputs = {k: v for k, v in list (batch[0].items())}
        batch_labels = batch[1]

        output = model(**batch_inputs)
        print(output)
        #         loss = CELoss(output.view(-1, output.size(-1)), batch_labels.view(-1))

#         optimizer.zero_grad()
#         loss.backward()

#         optimizer.step()

#         if (iteration + 1) % 10 == 0:
#             print(f'{iteration:3} - loss: {loss.item()}')

#     # todo 매 에포크가 끝나면 dev 데이터로 성능 비교하기
#     # Early Stopping 적용하기

each_len_list [27]


TypeError: 'int' object is not callable

In [331]:
for epoch in range(3): # 숫자 ->train_epoch
    model.train()
    
    for iteration, batch in enumerate(dataloader_train):
        batch_inputs = {k: v for k, v in list (batch[0].items())}
        batch_labels = batch[1]

        emb = bert(**batch_inputs)
        e = nn.Dropout(0.1)
        r = e(emb['last_hidden_state'])
        
        w = nn.Linear(768, 22)
        y = w(r)
        
        print(y)
        
#         q = nn.Softmax(dim = 2) # A dimension along which Softmax will be computed
#         output = y # A dimension along which Softmax will be computed
        
#         loss = CELoss(output.view(-1, output.size(-1)), batch_labels.view(-1))

#         optimizer.zero_grad()
#         loss.backward()

#         optimizer.step()

#         if (iteration + 1) % 10 == 0:
#             print(f'{iteration:3} - loss: {loss.item()}')

    # todo 매 에포크가 끝나면 dev 데이터로 성능 비교하기
    # Early Stopping 적용하기

each_len_list [83]
tensor([[[-0.2132,  0.0465, -0.0252,  ..., -0.5054, -0.3538,  0.1319],
         [ 0.1933,  0.0557,  0.1594,  ...,  0.2523,  0.1222, -0.1717],
         [ 0.1447,  0.3433,  0.3282,  ..., -0.2841, -0.0255,  0.1288],
         ...,
         [ 0.0983, -0.0163,  0.0341,  ..., -0.2113, -0.3903,  0.1640],
         [ 0.3565,  0.0075,  0.0812,  ..., -0.1527, -0.2726,  0.0307],
         [ 0.0881,  0.0353,  0.0372,  ...,  0.1172, -0.1950,  0.1636]]],
       grad_fn=<AddBackward0>)
each_len_list [14]
tensor([[[-0.3674, -0.2836,  0.0923, -0.1114,  0.0924,  0.2022,  0.2518,
          -0.1406, -0.3531,  0.0750,  0.3285, -0.0536,  0.3797, -0.0484,
           0.5099, -0.1962, -0.2210, -0.2339, -0.1866,  0.1348, -0.2580,
           0.3822],
         [ 0.0033,  0.3397,  0.2714, -0.0169,  0.0635, -0.2374, -0.0345,
          -0.3715, -0.0262,  0.1878,  0.3272,  0.3728,  0.3978,  0.1897,
          -0.1752,  0.0014,  0.2955, -0.3374,  0.5311,  0.3953,  0.3317,
           0.4161],
         [ 

KeyboardInterrupt: 

In [80]:
for epoch in range(5): # 숫자 ->train_epoch
    model.train()

    for data in dataloader_train:
        print(data['input'], data['label'])
#         loss = CELoss(data['input'].view(-1, output.size(-1)), data['label'].view(-1))

#         optimizer.zero_grad()
#         loss.backward()

#         optimizer.step()

#         if (iteration + 1) % 10 == 0:
#             print(f'{iteration:3} - loss: {loss.item()}')

    # todo 매 에포크가 끝나면 dev 데이터로 성능 비교하기
    # Early Stopping 적용하기

each_len_list [3]


ValueError: too many dimensions 'str'

# Mine

In [ ]:
for epoch in range(10): # 숫자 ->train_epoch
    model.train()

    for iteration, batch in enumerate(dataloader_train):
        batch_inputs = {k: v.cuda(device) for k, v in list(batch[0].items())}
        batch_labels = batch[1].cuda(device)

        output = model(**batch_inputs)
        loss = CELoss(output.view(-1, output.size(-1)), batch_labels.view(-1))

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        if (iteration + 1) % 10 == 0:
            print(f'{iteration:3} - loss: {loss.item()}')

    # todo 매 에포크가 끝나면 dev 데이터로 성능 비교하기
    # Early Stopping 적용하기

# Sample

In [ ]:
model.eval()

gold_list = []
pred_list = []

with torch.no_grad():
    for iteration, batch in enumerate(dataloader_test):
        batch_inputs = {k: v.cuda(device) for k, v in list(batch[0].items())}
        batch_labels = batch[1].cuda(device)
        
        output = model(**batch_inputs)
        loss = CELoss(output.view(-1, output.size(-1)), batch_labels.view(-1))
        
        print('loss:', loss.item())
        pred_ids = torch.argmax(output, dim=-1)
        
        for g, p in zip(batch_labels, pred_ids):
            gold_mask = g != tag_converter.pad_id
            
            gold = tag_converter.convert_ids_to_tags(g[gold_mask].tolist())
            pred = tag_converter.convert_ids_to_tags(p[gold_mask].tolist())
            gold_list.append(gold)
            pred_list.append(pred)
            
            print(gold)
            print(pred)

# Mine

In [226]:
model.eval()

gold_list = []
pred_list = []

with torch.no_grad():
    for iteration, batch in enumerate(dataloader_test):
        batch_inputs = {k: v for k, v in list(batch[0].items())}
        batch_labels = batch[1]
        
        output = model(**batch_inputs)
        loss = CELoss(output.view(-1, output.size(-1)), batch_labels.view(-1))
        
        print('loss:', loss.item())
        pred_ids = torch.argmax(output, dim=-1)
        
        for g, p in zip(batch_labels, pred_ids):
            gold_mask = g != tag_converter.pad_id
            
            gold = tag_converter.convert_ids_to_tags(g[gold_mask].tolist())
            pred = tag_converter.convert_ids_to_tags(p[gold_mask].tolist())
            gold_list.append(gold)
            pred_list.append(pred)
            
            print(gold)
            print(pred)

TypeError: train() takes 1 positional argument but 2 were given

In [ ]:
gold_list_flat = []
pred_list_flat = []
for g, p in zip(gold_list, pred_list):
    gold_list_flat += g
    pred_list_flat += p

In [ ]:
print(classification_report(gold_list_flat, pred_list_flat, digits=5, labels=list(tag_converter.tag_to_id.keys())[1:]))

In [ ]:
def get_chunk_type(tag_name):
    tag_class = tag_name.split('-')[0]
    tag_type = tag_name.split('-')[-1]
    return tag_class, tag_type

In [ ]:
def get_chunks(seq):
    default = "O"

    chunks = []
    chunk_type, chunk_start = None, None
    for i, tok in enumerate(seq):
        # End of a chunk 1
        if tok == default and chunk_type is not None:
            # Add a chunk.
            chunk = (chunk_type, chunk_start, i)
            chunks.append(chunk)
            chunk_type, chunk_start = None, None

        # End of a chunk + start of a chunk!
        elif tok != default:
            tok_chunk_class, tok_chunk_type = get_chunk_type(tok)
            if chunk_type is None:
                chunk_type, chunk_start = tok_chunk_type, i
            elif tok_chunk_type != chunk_type or tok_chunk_class == "B":
                chunk = (chunk_type, chunk_start, i)
                chunks.append(chunk)
                chunk_type, chunk_start = tok_chunk_type, i
        else:
            pass

    # end condition
    if chunk_type is not None:
        chunk = (chunk_type, chunk_start, len(seq))
        chunks.append(chunk)

    return chunks

In [ ]:
def evaluate_ner_F1(total_answers, total_preds):
    num_match = num_preds = num_answers = 0

    for answers, preds in zip(total_answers, total_preds):

        answer_seg_result = set(get_chunks(answers))
        pred_seg_result = set(get_chunks(preds))

        num_match += len(answer_seg_result & pred_seg_result)
        num_answers += len(answer_seg_result)
        num_preds += len(pred_seg_result)

    precision = 100.0 * num_match / num_preds
    recall = 100.0 * num_match / num_answers
    F1 = 2 * precision * recall / (precision + recall)

    return precision, recall, F1

In [ ]:
evaluate_ner_F1(gold_list, pred_list)